In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [4]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader=WebBaseLoader("https://developer.mozilla.org/en-US/docs/Learn/Tools_and_testing/Client-side_JavaScript_frameworks/React_getting_started")
loader

In [6]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://developer.mozilla.org/en-US/docs/Learn/Tools_and_testing/Client-side_JavaScript_frameworks/React_getting_started', 'title': 'Getting started with React - Learn web development | MDN', 'description': "This brings us to the end of our initial look at React, including how to install it locally, creating a starter app, and how the basics work. In the next article, we'll start building our first proper application — a todo list. Before we do that, however, let's recap some of the things we've learned.", 'language': 'en-US'}, page_content='Getting started with React - Learn web development | MDNSkip to main contentSkip to searchSkip to select languageMDN Web DocsOpen main menuReferencesReferencesOverview / Web TechnologyWeb technology reference for developersHTMLStructure of content on the webCSSCode used to describe document styleJavaScriptGeneral-purpose scripting languageHTTPProtocol for transmitting web resourcesWeb APIsInterfaces for building web a

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [9]:
documents

[Document(metadata={'source': 'https://developer.mozilla.org/en-US/docs/Learn/Tools_and_testing/Client-side_JavaScript_frameworks/React_getting_started', 'title': 'Getting started with React - Learn web development | MDN', 'description': "This brings us to the end of our initial look at React, including how to install it locally, creating a starter app, and how the basics work. In the next article, we'll start building our first proper application — a todo list. Before we do that, however, let's recap some of the things we've learned.", 'language': 'en-US'}, page_content='Getting started with React - Learn web development | MDNSkip to main contentSkip to searchSkip to select languageMDN Web DocsOpen main menuReferencesReferencesOverview / Web TechnologyWeb technology reference for developersHTMLStructure of content on the webCSSCode used to describe document styleJavaScriptGeneral-purpose scripting languageHTTPProtocol for transmitting web resourcesWeb APIsInterfaces for building web a

In [30]:
from langchain_community.llms import Ollama

In [31]:
llm=Ollama(model="gemma:2b")

C:\Users\Dell\AppData\Local\Temp\ipykernel_26760\168757482.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="gemma:2b")


In [32]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings=(
    OllamaEmbeddings(model="gemma:2b")  ##by default it ues llama2
)


C:\Users\Dell\AppData\Local\Temp\ipykernel_26760\1648917218.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  OllamaEmbeddings(model="gemma:2b")  ##by default it ues llama2


In [33]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [34]:
vectorstoredb

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| Ollama(model='gemma:2b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [36]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [39]:
response=retrieval_chain.invoke({"input":"React does not enforce strict rules"})
response['answer']

"Sure, here's the answer to the question:\n\nThe context provides an overview of React, including its purpose as a library for building user interfaces, its independence from frameworks, and its use with other libraries. It also emphasizes the benefits and drawbacks of using React compared to other frameworks.\n\nThe context then provides a brief description of React and how it is used. It also mentions the importance of JSX and how it is used in React.\n\nThe context concludes by stating that React can be used for small pieces of an interface, but it is more approachable when used in conjunction with other libraries like Vue or jQuery."

In [40]:
response

{'input': 'React does not enforce strict rules',
 'context': [Document(metadata={'source': 'https://developer.mozilla.org/en-US/docs/Learn/Tools_and_testing/Client-side_JavaScript_frameworks/React_getting_started', 'title': 'Getting started with React - Learn web development | MDN', 'description': "This brings us to the end of our initial look at React, including how to install it locally, creating a starter app, and how the basics work. In the next article, we'll start building our first proper application — a todo list. Before we do that, however, let's recap some of the things we've learned.", 'language': 'en-US'}, page_content='Hello ReactAs its official tagline states, React is a library for building user interfaces. React is not a framework – it\'s not even exclusive to the web. It\'s used with other libraries to render to certain environments. For instance, React Native can be used to build mobile applications.\nTo build for the web, developers use React in tandem with ReactDOM.